In [225]:
import pandas as pd
import numpy as np
import datetime

In [226]:
#Leo la data proveniente de un csv
path = 'datos/installs.csv'
installs = pd.read_csv(path)

In [227]:
# Damos formato a la fecha.
installs['timestamp_format'] = pd.to_datetime(installs['created'])
# Agrego columna fecha
installs['date_timestamp'] = installs['timestamp_format'].dt.date
# Agrego columna numero dia.
installs['day'] = installs['timestamp_format'].dt.day

In [228]:
# ventanas de 3 dias
installs18_20 = installs[(installs['day'] >= 18) & (installs['day'] < 21)]
installs19_21 = installs[(installs['day'] >= 19) & (installs['day'] < 22)]
installs20_22 = installs[(installs['day'] >= 20) & (installs['day'] < 23)]
installs21_23 = installs[(installs['day'] >= 21) & (installs['day'] < 24)]
installs22_24 = installs[(installs['day'] >= 22) & (installs['day'] < 25)]
installs23_25 = installs[(installs['day'] >= 23) & (installs['day'] < 26)]
installs24_26 = installs[(installs['day'] >= 24) & (installs['day'] < 27)]

In [229]:
# Agregamos variable a predecir a los dataframes
installs18_20['base_time'] = datetime.datetime(2019, 4, 18)
installs18_20['time_to_install'] = (installs18_20['timestamp_format'] - installs18_20['base_time']) / np.timedelta64(1, 's')

installs19_21['base_time'] = datetime.datetime(2019, 4, 19)
installs19_21['time_to_install'] = (installs19_21['timestamp_format'] - installs19_21['base_time']) / np.timedelta64(1, 's')

installs20_22['base_time'] = datetime.datetime(2019, 4, 20)
installs20_22['time_to_install'] = (installs20_22['timestamp_format'] - installs20_22['base_time']) / np.timedelta64(1, 's')

installs21_23['base_time'] = datetime.datetime(2019, 4, 21)
installs21_23['time_to_install'] = (installs21_23['timestamp_format'] - installs21_23['base_time']) / np.timedelta64(1, 's')

installs22_24['base_time'] = datetime.datetime(2019, 4, 22)
installs22_24['time_to_install'] = (installs22_24['timestamp_format'] - installs22_24['base_time']) / np.timedelta64(1, 's')

installs23_25['base_time'] = datetime.datetime(2019, 4, 23)
installs23_25['time_to_install'] = (installs23_25['timestamp_format'] - installs23_25['base_time']) / np.timedelta64(1, 's')

installs24_26['base_time'] = datetime.datetime(2019, 4, 24)
installs24_26['time_to_install'] = (installs24_26['timestamp_format'] - installs24_26['base_time']) / np.timedelta64(1, 's')

/home/santiago/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/santiago/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/santiago/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

# Feature Engineering

## Cantidad de Instalaciones

In [251]:
final = installs18_20.groupby('ref_hash')['created'].count().reset_index()
final.columns = ['ref_hash', 'cant_installs']
final2 = installs21_23.groupby('ref_hash')['created'].count().reset_index()
final2.columns = ['ref_hash', 'cant_installs']

## Marca de celular del user

In [252]:
brand_user = installs18_20.groupby('ref_hash').agg({'device_brand': 'first'}).reset_index()
brand_user.columns = ['ref_hash', 'device_brand']

final = final.merge(brand_user, on='ref_hash', how='left')
final['device_brand'].fillna(0, inplace= True)

In [253]:
brand_user2 = installs21_23.groupby('ref_hash').agg({'device_brand': 'first'}).reset_index()
brand_user2.columns = ['ref_hash', 'device_brand']

final2 = final2.merge(brand_user2, on='ref_hash', how='left')
final2['device_brand'].fillna(0, inplace= True)

## Modelo de celular del user

In [254]:
modelo_user = installs18_20.groupby('ref_hash').agg({'device_model': 'first'}).reset_index()
modelo_user.columns = ['ref_hash', 'device_model']

final = final.merge(modelo_user, on='ref_hash', how='left')
final['device_model'].fillna(0, inplace= True)

In [255]:
modelo_user2 = installs21_23.groupby('ref_hash').agg({'device_model': 'first'}).reset_index()
modelo_user2.columns = ['ref_hash', 'device_model']

final2 = final2.merge(modelo_user2, on='ref_hash', how='left')
final2['device_model'].fillna(0, inplace= True)

## Id de la aplicacion instalada

In [256]:
app_id = installs18_20.groupby('ref_hash').agg({'application_id': 'last'}).reset_index()
app_id.columns = ['ref_hash', 'application_id']

final = final.merge(app_id, on='ref_hash', how='left')
final['application_id'].fillna(0, inplace= True)

In [257]:
app_id2 = installs21_23.groupby('ref_hash').agg({'application_id': 'last'}).reset_index()
app_id2.columns = ['ref_hash', 'application_id']

final2 = final2.merge(app_id2, on='ref_hash', how='left')
final2['application_id'].fillna(0, inplace= True)

## Cantidad de aplicaciones distintas instaladas por user

In [258]:
cantidad_de_apps = installs18_20.groupby('ref_hash')['application_id'].nunique().reset_index()
cantidad_de_apps.columns = ['ref_hash', 'cant_apps']

final = final.merge(cantidad_de_apps, on='ref_hash', how='left')
final['cant_apps'].fillna(0, inplace= True)

In [259]:
cantidad_de_apps2 = installs21_23.groupby('ref_hash')['application_id'].nunique().reset_index()
cantidad_de_apps2.columns = ['ref_hash', 'cant_apps']

final2 = final2.merge(cantidad_de_apps2, on='ref_hash', how='left')
final2['cant_apps'].fillna(0, inplace= True)

## User agent

In [260]:
user_agent = installs18_20.groupby('ref_hash').agg({'user_agent': 'last'}).reset_index()
user_agent.columns = ['ref_hash', 'user_agent']

final = final.merge(user_agent, on='ref_hash', how='left')

In [261]:
user_agent2 = installs21_23.groupby('ref_hash').agg({'user_agent': 'last'}).reset_index()
user_agent2.columns = ['ref_hash', 'user_agent']

final2 = final2.merge(user_agent2, on='ref_hash', how='left')

## Device Language

In [262]:
device_language = installs18_20.groupby('ref_hash').agg({'device_language': 'last'}).reset_index()
device_language.columns = ['ref_hash', 'device_language']

final = final.merge(device_language, on='ref_hash', how='left')

In [263]:
device_language2 = installs21_23.groupby('ref_hash').agg({'device_language': 'last'}).reset_index()
device_language2.columns = ['ref_hash', 'device_language']

final2 = final2.merge(device_language2, on='ref_hash', how='left')

## Join de features de clicks

In [264]:
clicks = pd.read_csv('features_clicks/clicks_18_20.csv')
clicks2 = pd.read_csv('features_clicks/clicks_21_23.csv')

In [265]:
final = final.merge(clicks, on='ref_hash', how='left')
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132297 entries, 0 to 132296
Data columns (total 14 columns):
ref_hash               132297 non-null int64
cant_installs          132297 non-null int64
device_brand           132297 non-null float64
device_model           132297 non-null float64
application_id         132297 non-null int64
cant_apps              132297 non-null int64
user_agent             92537 non-null object
device_language        126712 non-null float64
cant_clicks            1731 non-null float64
cant_wifi_clicks       1731 non-null float64
mean_time_to_click     1731 non-null float64
cant_sources_clicks    1731 non-null float64
cant_advs_clicks       1731 non-null float64
mean_location          1731 non-null object
dtypes: float64(8), int64(4), object(2)
memory usage: 15.1+ MB


In [266]:
final2 = final2.merge(clicks2, on='ref_hash', how='left')

## TimeToInstall -> variable a predecir

In [267]:
time_to_install = installs18_20.groupby('ref_hash').agg({'time_to_install': 'mean'}).reset_index()
time_to_install.columns = ['ref_hash', 'time_to_install']

# 3 dias son 259200 segundos
final = final.merge(time_to_install, on='ref_hash', how='left')
final['time_to_install'].fillna(259200, inplace= True)

In [268]:
time_to_install2 = installs21_23.groupby('ref_hash').agg({'time_to_install': 'mean'}).reset_index()
time_to_install2.columns = ['ref_hash', 'time_to_install']

# 3 dias son 259200 segundos
final2 = final2.merge(time_to_install2, on='ref_hash', how='left')
final2['time_to_install'].fillna(259200, inplace= True)

In [269]:
# 3 dias son 259200 segundos
final['time_to_install'] = final['time_to_install'].fillna(259200)
final2['time_to_install'] = final2['time_to_install'].fillna(259200)

In [270]:
final.to_csv('test_and_train_installs/data_18_20.csv', index=False)
final2.to_csv('test_and_train_installs/data_21_23.csv', index=False)

## Join de targets con features

In [141]:
#targets = pd.read_csv('datos/target_competencia_ids.csv')

In [142]:
#final['ref_hash'] = final['ref_hash'].astype(str) + '_sc'
#final['ref_hash'] = final['ref_hash'].astype('object')

#merged_18_20_targets = targets.merge(final, on='ref_hash', how='left').drop('obj', axis=1)
#merged_18_20_targets = merged_18_20_targets.loc[merged_18_20_targets['ref_hash'].str.contains('_sc'), :]

# 3 dias son 259200 segundos
#merged_18_20_targets['time_to_install'] = merged_18_20_targets['time_to_install'].fillna(259200)

In [143]:
#final2['ref_hash'] = final2['ref_hash'].astype(str) + '_sc'
#final2['ref_hash'] = final2['ref_hash'].astype('object')

#merged_21_23_targets = targets.merge(final2, on='ref_hash', how='left').drop('obj', axis=1)
#merged_21_23_targets = merged_21_23_targets.loc[merged_21_23_targets['ref_hash'].str.contains('_sc'), :]

# 3 dias son 259200 segundos
#merged_21_23_targets['time_to_install'] = merged_21_23_targets['time_to_install'].fillna(259200)

In [146]:
#merged_18_20_targets.to_csv('test_and_train_installs/data_18_20.csv', index=False)
#merged_21_23_targets.to_csv('test_and_train_installs/data_21_23.csv', index=False)